### Data Wrangling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json


In [ ]:
pv_df_orig = pd.read_csv('Capstone/openpv_all.csv', sep=',', low_memory=False)


In [ ]:
pv_df_orig.info()

In [ ]:
# FUNCTION: remove NULL columns
def remove_null_col(df, percent=80):
  '''Remove all columns with too many null values'''
  keeper_list = []
  # loser_list = []
  col_list = list(df.columns)
  # iterate through each column
  for col_name in col_list:
    # A and B are the conditions for removing columns
    nan_count = pd.isnull(df[col_name]).sum()
    A = df[col_name].isnull().all()
    B = nan_count/len(df.index)*100 > percent
    # if column is empty or more than 20% of columns is NaN, then discard, else keep
    if (A | B):
      # place column into a keeper list
      # loser_list.append(col_name)
      print('column removed:', col_name)
    else:
      keeper_list.append(col_name)
  #pv_df = df[keeper_list]
  return df[keeper_list]
pv_df = remove_null_col(pv_df_orig, 80.0)

# NOTE: use dropna(axis=1, how='all') to drop entire column
pv_df = pv_df[['date_installed', 'city','state','zipcode', 'county', 'size_kw','annual_insolation',
               'annual_PV_prod', 'reported_annual_energy_prod',
               'cost_per_watt','cost', 'sales_tax_cost', 'rebate', 'incentive_prog_names', 'install_type',
               'installer','utility_clean']]


In [ ]:
# CONVERT DATA TYPE OF 'ZIPCODES'
# convert to numpy array, replace missing values with 0, then convert data type
pv_df['zipcode'] = np.nan_to_num(pv_df['zipcode'].values).astype(int)


def missing_val_count(df, col_name, show_missing=False):
  '''Count missing values and show its index.'''
  print('\nColumn:', col_name, '\nMissing values:', np.count_nonzero(df.loc[:, col_name].isnull().values))
  if show_missing == True:
    print(df[df.loc[:, col_name].isnull()])
print(missing_val_count(pv_df, 'state'))


def drop_n_reset(df):
  '''Drop all missing rows, duplicates, and reset the index.'''
  df = df.dropna(axis=0, how='all').drop_duplicates()
  df = df.reset_index(drop=True)
drop_n_reset(pv_df)


### CONVERT DATA TYPES AND REMOVE STRINGS/SYMBOLS

In [ ]:
# NUMBER OF STATES
print(pv_df.state.nunique()) # number of unique states
print(pv_df.state.unique()) # OH, MD and PA contain an empty string, PR and DC are not states

In [ ]:
# NOTE: cost and rebate columns are dtype 'object' although they are supposed to be numeric
# test for any digits within character strings
col = list(pv_df.columns)
for item in col:
  if pv_df[item].dtype == np.object:
      if pv_df[item].str.isnumeric().any():
        print(item, '<- dtype is object but contains digits')



Cost and rebate do not convert due to error. Strings must be replaced before dtype conversion

In [ ]:
pv_df.loc[:, 'cost'] = pv_df.loc[:, 'cost'].values.astype(float)
pv_df.loc[:, 'rebate'] = pv_df.loc[:, 'rebate'].values.astype(float)


In [ ]:
# FUNCTION: look for string 'n/a', dollar sign '$', string 'null', and comma ','
# strip all leading and trailing whitespaces
def remove_symbols(df):
  '''For columns which are dtype 'object', strip all leading and trailing whitespaces
  remove unwanted strings/symbols such as dollar sign and comma used in currency.
  Replace the string 'n/a' and 'null' with a numeric zero'''
  cols = list(df.columns)
  for item in cols:
    if df.loc[:, item].dtype == np.object:
      df.loc[:, item] = df.loc[:, item].str.strip('$')
      df.loc[:, item] = df.loc[:, item].replace(['n/a', 'null'], 0)
    # target specific columns
    elif item == 'state':
      df.loc[:, 'state'] = df.loc[:, 'state'].str.rstrip()
  return df
pv_df = remove_symbols(pv_df)

pv_df.loc[:, 'rebate'] = pv_df.loc[:, 'rebate'].str.replace(',', '')


In [ ]:
# confirm removal of unwanted characters
conditions = [pv_df['cost'].str.contains('n/a').any(), pv_df['rebate'].str.contains('\$').any(),
              pv_df['rebate'].str.contains(',').any(), pv_df['rebate'].str.contains('null').any()]
# check for conditions being satisfied
for item in conditions:
  if item:
    print(conditions)
    print("Test failed. You're not done")


In [ ]:
# convert zeros to NaNs since cost cannot be zero
pv_df.loc[:, 'cost']= pv_df.loc[:, 'cost'].replace(0, np.nan)

# CONVERT TO FLOAT
pv_df.loc[:, 'cost'] = pv_df.loc[:, 'cost'].values.astype(float)
pv_df.loc[:, 'rebate'] = pv_df.loc[:, 'rebate'].values.astype(float)


### LOWER CASE
Convert data types prior to Lower Case

In [ ]:

# create function to to test data type and convert to lower case
def upper_to_lower(df):
  ''' Test each column for data type 'object', then convert to lower case '''

  col_name = list(df.columns)
  # iterate over columns
  for item in col_name:
    # if data type is object, then convert column to lower case
    if df[item].dtype == np.object:
      df.loc[:, item] = df.loc[:, item].str.lower()
      print(item, '-converted to lower case')
  return df
pv_df = upper_to_lower(pv_df)


### ZIPCODES

Zipcode column contains to many three and four digit zipcodes, some of which turned out to be invalid. This column will be removed.

In [ ]:

# create missing values in 'state' column then drop rows
pv_df[pv_df['zipcode']==0]=np.nan
# pv_df['zipcode'] = np.nan_to_num(pv_df['zipcode'].values).astype(int)

# count missing values created in the 'state' column
missing_val_count(pv_df, 'state', True)

# pv_df.info()
# drop NA rows, duplicates and reset index
pv_df = pv_df.dropna(axis=0, how='all').drop_duplicates().reset_index(drop=True)

# NOTE: some zipcodes have only 3 or 4 digits with missing zeros. Consider REMOVING
pv_df = pv_df.sort_values(by = ['zipcode'])
pv_df.loc[1950:2000, 'zipcode']


Some zipcodes have only 3 or 4 digits with missing leading zeros as a result they will be removed.

In [ ]:
# select columns to work with further
pv_df = pv_df[['date_installed', 'city','state', 'county', 'size_kw','annual_insolation',
               'annual_PV_prod', 'reported_annual_energy_prod',
               'cost_per_watt','cost','rebate', 'incentive_prog_names', 'install_type',
               'installer','utility_clean']]


pv_df = pv_df.dropna(axis=0, how='all').drop_duplicates().reset_index(drop=True)


In [ ]:
# found character '\r' within 'city' column and removed it
pv_df['city'].str.contains('\r').any()
pv_df['city'] = pv_df['city'].str.strip('\r')



### APIs


In [ ]:
# =============================================================================
# APIs
# =============================================================================

# CREATE ADDRESS
# create city, state combo for use in the APIs
df = pv_df[['city', 'state', 'county']]
df = pv_df[['state', 'county']].drop_duplicates()
len(df)
# concat city and state; county and state
# create an address list
county_state_ls = list(df['county'] + ', ' + df['state'])
len(county_state_ls)
# create a state list
state_ls = list(pv_df['state'].unique())
len(state_ls)


# =============================================================================
# SOLAR RADIATION
# =============================================================================
cities = []
states = []
solrad_ = []
url = 'https://developer.nrel.gov/api/pvwatts/v5.json?api_key={API_KEY}&address={address}&system_capacity={sys_cap}&azimuth={azimuth}&tilt={tilt}&array_type={tracking}&module_type=1&losses=10'
for address in county_state_ls:
  url_new = url.format(API_KEY='0DkoSjFm3OJ21FRtM05Smfi9bPNoFRcJHpFNgNJw',address=address, sys_cap=sys_cap, azimuth=azimuth, tilt=tilt, tracking=tracking)
  # Package the request, send the request and catch the response: r
  req = requests.get(url_new)
  programs_data = req.json()
  try:
    city = programs_data['station_info']['city']
    state = programs_data['station_info']['state']
    solrad = programs_data['outputs']['solrad_annual']
    cities.append(city)
    states.append(state)
    solrad_.append(solrad)
  except:
    pass

sol_df = pd.DataFrame({'cities': cities, 'states':states, 'solar_rad':solrad_})
# clean df; drop duplicates, remove underscore, and reset index
sol_df = sol_df.drop_duplicates()
sol_df['cities'] = sol_df['cities'].str.replace('_', ' ')
sol_df = sol_df.reset_index(drop=True)

# convert to lower case
sol_df = upper_to_lower(sol_df)

# write file
# sol_df.to_csv('solar_rad_state.csv', encoding='utf-8', index=False)


In [ ]:
# =============================================================================
# POPULATION OF CITIES
# =============================================================================
cities = []
states = []
population = []
for i in range(999):
  url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=1000-largest-us-cities-by-population-with-geographic-coordinates&rows=1000&sort=-rank&facet=city&facet=state'
  req = requests.get(url)
  pop_data = req.json()
  city = pop_data['records'][i]['fields']['city']
  state = pop_data['records'][i]['fields']['state']
  pop = pop_data['records'][i]['fields']['population']
  cities.append(city)
  states.append(state)
  population.append(pop)

pop_df = pd.DataFrame({'cities': cities, 'states':states, 'population':population})
# lower case
pop_df = upper_to_lower(pop_df)

# wrtie to csv
pop_df.to_csv('population_df.csv', index=False)
# read csv
pop_df = pd.read_csv('population_df.csv', sep=',', low_memory=False)
pop_df['states'].unique()
pop_df['cities'].nunique()
pop_df[pop_df['states']=='district of columbia']



In [ ]:
# =============================================================================
# INCENTIVES
# =============================================================================

incentive_missing = pv_pop[pv_pop['incentive_prog_names'].isnull()]
incentives = incentive_missing[['city', 'state', 'incentive_prog_names']].drop_duplicates()
address_list = list(incentives['city'] + ', ' + incentives['state'])

# extract the name of program and rebate amount
state = []
incentive = []
for item in address_list:
  url = 'https://developer.nrel.gov/api/energy_incentives/v2/dsire.json?api_key={API_KEY}&address={address}&category=solar_technologies&technology=solar_photovoltaics'
  url_new = url.format(API_KEY='0DkoSjFm3OJ21FRtM05Smfi9bPNoFRcJHpFNgNJw',address=item)
  req = requests.get(url_new)
  programs_data = req.json()

  try:
    # iterate over each program
    num_prog = len(programs_data['result'])
    for i in range(num_prog):
      state_name = (programs_data['result'][i]['regions'][0]['name']).lower()
      incentive_name = programs_data['result'][i]['program_name']
      regions = programs_data['result'][i]['regions'][0]['type']

      # extract info: state, and incentive program for state
      # if program is for the state (not federal), append info to list
      if regions == 'state':
        state.append(state_name)
        incentive.append(incentive_name)
  except:
    pass

incentive_df = pd.DataFrame({'state': state, 'incentive_program': incentive})
incentive_df = incentive_df.drop_duplicates().reset_index(drop=True)
# lower case
incentive_df['incentive_program'] = incentive_df['incentive_program'].str.lower()
# write file
incentive_df.to_csv('incentives_state.csv', index=False)


### Merge dataframes obtained from APIs

In [ ]:
def load_csv(file_name):
  df = pd.read_csv(file_name, sep=',', low_memory=False)
  return df

# open files
sol_df = load_csv('solar_rad_state.csv')
incent_df = load_csv('incentives_state.csv')
states_abbv = load_csv('states_abbreviation.csv')
pop_df = load_csv('population_df.csv')

# merge pop_df and pv_df, MERGE on STATES
pop_df.head()
states_abbv.head()
pv_df2 = pd.merge(pv_df, states_abbv, left_on='state', right_on='abbreviation', how='left').dropna(axis=0, how='all').drop_duplicates()
pv_df2 = pv_df2.reset_index(drop=True)
pv_df2.head()
pv_df2 = pv_df2[['date_installed', 'city','state', 'full', 'county', 'size_kw','annual_insolation',
               'annual_PV_prod', 'reported_annual_energy_prod','cost_per_watt','cost','sales_tax_cost','rebate',
               'incentive_prog_names', 'install_type', 'installer','utility_clean']]
pv_df2.info()
pv_df.info()

# merge population
pv_df3 = pd.merge(pv_df2, pop_df, left_on=['city','full'], right_on=['cities','states'], how='left')
pv_df3 = pv_df3.dropna(axis=0, how='all').drop_duplicates().reset_index(drop=True)
pv_df3.info()
pv_df3.head()
pv_df3 = pv_df3[['date_installed', 'city','state', 'full', 'county', 'population', 'size_kw','annual_insolation',
               'annual_PV_prod', 'reported_annual_energy_prod','cost_per_watt','cost', 'sales_tax_cost', 'rebate',
               'incentive_prog_names', 'install_type', 'installer','utility_clean']]



In [ ]:
pv_df = pv_df3
# rename columns
pv_df3.columns = ['date_installed', 'city', 'state_short', 'state', 'county', 'population',
                 'size_kw', 'annual_insolation', 'annual_pv_prod',
                 'reported_annual_energy_prod', 'cost_per_watt', 'cost','sales_tax_cost', 'rebate',
                 'incentive_prog_names', 'install_type', 'installer', 'utility']



In [ ]:
pv_df['county'].str.contains('st. louis').any()
pv_df['county'].str.contains('st louis').any()
# found both: 'st. louis' and 'st louis' in county column
pv_df[pv_df['county']=='st. louis']
pv_df[pv_df['state']=='mo']

# remove the period from 'st.' to maintain consistency
pv_df['city'] = pv_df['city'].str.replace('.', '')
pv_df['county'] = pv_df['county'].str.replace('.', '')


In [ ]:
# =============================================================================
# SIZE KW
# =============================================================================
# show summary statistics
pv_df.size_kw.describe()
missing_val_count(pv_df, 'size_kw', True)


In [ ]:
# =============================================================================
# INCENTIVE_PROG_NAMES
# =============================================================================
# explore the missing values
missing_val_count(pv_df, 'incentive_prog_names', True)
pv_df.incentive_prog_names.describe()
pv_df.incentive_prog_names.unique()

# unique values
pv_df['incentive_prog_names'].nunique()

# frequency count of programs
pv_df.groupby(['incentive_prog_names'])['state'].value_counts().sort_values()


# REBATE for each state
# median rebate offered by each program
pv_df.groupby(['incentive_prog_names'])['rebate'].median().sort_values()

pv_df.groupby(['state'])['rebate'].median().sort_values()
pv_df.groupby(['state'])['cost'].median().sort_values()


In [ ]:
# =============================================================================
# MISSING VALUES: cost
# =============================================================================
missing_val_count(pv_df, 'cost', True) # 266,670
pv_df['cost'].describe()


In [ ]:
# =============================================================================
# # SUMMARY STATS: cost_per_watt
# =============================================================================
pv_df['cost_per_watt'].describe()
missing_val_count(pv_df, 'cost_per_watt', True) # 266,914
missing_val_count(pv_df, 'size_kw', True)


In [ ]:
# explore the columns
pv_df[['cost_per_watt', 'cost', 'size_kw', 'annual_pv_prod']]


In [ ]:
# =============================================================================
# MISSING VALUES: cost_per_watt
# =============================================================================

# FILL COST_PER_WATT
# cost per watt = cost / kw*1000
# fill in missing COST_PER_WATT based on cost and size
pv_df['cost_per_watt'] = pv_df['cost_per_watt'].fillna(pv_df['cost'] / (pv_df['size_kw']*1000))

# FILL COST
# cost = cost_per_watt * (size*1000)
# fill in missing COST values based on SIZE and COST_PER_WATT
fill_cost = (pv_df['cost_per_watt']*(pv_df['size_kw']*1000))
pv_df['cost'] = pv_df['cost'].fillna(fill_cost)

cost = pv_df['cost'] / (pv_df['size_kw']*1000)
cost.describe()


In [ ]:
# =============================================================================
# # SUMMARY STATS: annual_PV_prod (estimated production)
# =============================================================================

# having all values for 'size_kw', fill in missing values in 'annual_pv_prod'
pv_df['annual_pv_prod'].describe()
missing_val_count(pv_df, 'annual_pv_prod',True) # 226010
pv_df[['annual_pv_prod', 'size_kw', 'annual_insolation']]


# annual energy production = annual_pv_produced/system_size
common_denom = pv_df['annual_pv_prod']/pv_df['size_kw']
common_denom.describe()
np.count_nonzero(common_denom.isnull().values) # 239608

# fill missing values within common_denom using its median
common_denom = common_denom.fillna(common_denom.median())

# infered annual production = common_denom*size
annual_pv_infer = common_denom*pv_df['size_kw']
annual_pv_infer.describe()

# check the error between actual and calculated energy production
error = abs(annual_pv_infer - pv_df['annual_pv_prod'])/pv_df['annual_pv_prod']
error.describe()
# fill missing values
pv_df['annual_pv_prod'] = pv_df['annual_pv_prod'].fillna(annual_pv_infer)
pv_df['annual_pv_prod'].describe()


In [ ]:
# =============================================================================
# SUMMARY STATS: reported_annual_energy_prod
# =============================================================================
# count and explore missing values
# 833,466 missing values
missing_val_count(pv_df, 'reported_annual_energy_prod', True)
# this may be too many missing values to fill as it may lead to a lrager error

pv_df['reported_annual_energy_prod'].describe()

pv_df[['reported_annual_energy_prod', 'annual_pv_prod']]


In [ ]:
# =============================================================================
# MISSING VALUES: reported_annual_energy_prod
# =============================================================================

# calculate error between estimated and reported annual PV production
error = abs(pv_df['reported_annual_energy_prod'] - pv_df['annual_pv_prod'])/pv_df['annual_pv_prod']
error.describe()
error.mean() # 0.1083
error.median() # 0.0691

# fill REPORTED_ANNUAL_ENERGY_PROD missing values based on error from estimated ANNUAL_PV_PROD
infer_reported = pv_df['annual_pv_prod']*error.mean()
pv_df['reported_annual_energy_prod'] = pv_df['reported_annual_energy_prod'].fillna(infer_reported)
pv_df['reported_annual_energy_prod'].describe()


In [ ]:
# =============================================================================
# SUMMARY STATS: annual_insolation
# use API to fill missing values
# =============================================================================
missing_val_count(pv_df, 'annual_insolation')
pv_df['annual_insolation'].describe()

# explore: insolation rate for each state
# insolation too high for some states: DO NOT USE
state_insol = pv_df.groupby('state')['annual_insolation'].mean()
state_insol.sort_values()
state_insol.describe()


In [ ]:
# =============================================================================
# install type, consolidate
# =============================================================================
pv_df['install_type'].unique()

pv_df[pv_df['install_type']=="customer"]
pv_df[pv_df['install_type']=="unknown"]
pv_df['install_type'].str.contains("gov't/np").value_counts()

# consolidate all commercial type
comr_ls = ['commerical','commercial - agriculture', 'small business', 'commercial - small business',
               'commercial - builders', 'commercial - other', 'commercial']
pv_df['install_type'] = pv_df['install_type'].replace(comr_ls, 'commercial')

# government
pv_df['install_type'] = pv_df['install_type'].replace("gov't/np", 'government')

# educational
pv_df['install_type'] = pv_df['install_type'].replace("education", 'educational')

# agricultural
pv_df['install_type'] = pv_df['install_type'].replace("agriculture", 'agricultural')

# residential
pv_df['install_type'] = pv_df['install_type'].replace('residential/sf', 'residential')

# residential
pv_df['install_type'] = pv_df['install_type'].replace('not stated', 'unknown')

# view the missing values
pv_df[pv_df['install_type'].isnull()]

# fill missing values with 'unknown'
pv_df['install_type'] = pv_df['install_type'].fillna('unknown')

pv_df['install_type'] = pv_df['install_type'].astype('category')


In [ ]:
# fills in missing values for annual solar radiation using the dataframe created through an API for annual 
# solar radiation as a refrence
def reference_fill(df_target, col1, col2, df_ref, col3, col4):
  '''Fill in missing values in df_target based on known values from df_ref column'''
  # iterate over the known values in reference column
  ls = list(df_ref[col3])
  for item in df_ref[col3]:
    # 1. REFERENCE: find position of unique item in column
    # 2. REFERENCE: extract row and contents
    # 3. TARGET: find position to fill
    # 4. TARGET: fill in value in found position with reference value
    
    # REFERENCE
    try:
      row = df_ref[(df_ref[col3]==item)].index[0]
      name = df_ref.loc[row, col4] # replacement value
      # TARGET
      rows = df_target[(df_target.loc[:, col1]==item)].index
      df_target.loc[rows, col2] = df_target.loc[rows, col2].fillna(name)
    except:
      pass


In [ ]:
# INSOLATION
sol_df.head()
pv_df.head()

reference_fill(pv_df, 'state_short', 'annual_insolation', sol_df, 'states', 'solar_rad')
missing_val_count(pv_df, 'annual_insolation', True) # 225,394
pv_df['annual_insolation'].describe()

# missing city for capital, DC
# fill with 'washington'
missing_val_count(pv_df, 'annual_insolation', True) # missing only in DC
pv_df[pv_df['state_short']=='dc']
rows = pv_df[(pv_df.loc[:, 'state_short']=='dc')].index
pv_df.loc[rows, 'city'] = pv_df.loc[rows, 'city'].fillna('washington')


sol_df[sol_df['states']=='dc']
sol_df['states'].nunique()
sol_df[sol_df['states']=='va']

# fill 'insolation' for DC with VA's
rows = pv_df[(pv_df.loc[:, 'state_short']=='dc')].index
pv_df.loc[rows, 'annual_insolation'] = pv_df.loc[rows, 'annual_insolation'].fillna(4.6741948)
missing_val_count(pv_df, 'annual_insolation')

In [ ]:
pv_df['population'].describe()
# median population
pv_df['population'].median()

# filter DF, filter by population
pv_pop = pv_df[pv_df['population']>3.688800e+04]
pv_pop = pv_pop.reset_index(drop=True)
pv_pop.info()

# 'installer' contains mistakes
pv_pop['installer'].unique()
pv_pop['installer'] = pv_pop['installer'].fillna('unknown')

pv_pop['utility'].unique()
pv_pop['utility'] = pv_pop['utility'].fillna('unknown')


In [ ]:
# count missing values
missing_val_count(pv_pop, 'incentive_prog_names') # 703
missing_val_count(pv_pop, 'annual_insolation') # 6943

In [ ]:
# fill in incentives' missing values
# use incent_df as a reference to fill in missing values of incentives for state
reference_fill(pv_pop, 'state', 'incentive_prog_names', incent_df, 'state', 'incentive_program')


In [ ]:
# INSOLATION
sol_df.head()
pv_df.head()

reference_fill(pv_pop, 'state_short', 'annual_insolation', sol_df, 'states', 'solar_rad')
missing_val_count(pv_pop, 'annual_insolation', True)
pv_df['annual_insolation'].describe()

# missing city for washington, DC
# fill with 'washington'
missing_val_count(pv_df, 'annual_insolation', True) # missing only in DC
pv_df[pv_df['state_short']=='dc']
rows = pv_df[(pv_df.loc[:, 'state_short']=='dc')].index
pv_df.loc[rows, 'city'] = pv_df.loc[rows, 'city'].fillna('washington')

# use neighbor's value for annual insolation
sol_df[sol_df['states']=='dc']
sol_df['states'].nunique()
sol_df[sol_df['states']=='va']

# fill 'insolation' for DC with VA's
rows = pv_df[(pv_df.loc[:, 'state_short']=='dc')].index
pv_df.loc[rows, 'annual_insolation'] = pv_df.loc[rows, 'annual_insolation'].fillna(4.6741948)
# check again for missing values
missing_val_count(pv_df, 'annual_insolation')


In [ ]:
# write file
# =============================================================================
pv_pop.to_csv('pv_pop_clean.csv', encoding='utf-8', na_rep='NA', index=False)
